In [117]:
import jieba
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [118]:
def error(true, pred):
    error = np.square(np.subtract(true,pred)).mean(axis=0)
    return error

In [119]:
df = pd.read_excel("ml-weibo-project-/raw_data.xlsx",skiprows=1,usecols=["微博正文","点赞数","转发数","评论数"])
df = df.rename(columns = {'微博正文':'text', '点赞数':'like', '转发数':'comment','评论数':'forward'}, inplace=False)

In [120]:
df.head()

,text,like,comment,forward
0,【#你好，明天#】岁末年初，即便你的年终盘点一言难尽，但新年的flag仍然计日可期。需要抓住...,5389,1997,406
1,【今天，发条微博，向这些“闪亮的名字”致敬】南仁东、林俊德、张超、王继才、黄群、宋月才、姜开...,6826,1268,289
2,【夜读：这一年，谢谢自己】转眼间，2018年就要过去了。有时候我们很怕，怕自己的成长速度跟不...,6565,4326,580
3,【小调查：跨年夜，你如何度过的？】#2018最后一天#，你是如何度过的？陪在家人身边？宅在家...,919,2562,5281
4,【揭秘！习主席书架上的新变化】今晚，习近平主席发表了#2019新年贺词#。这一次，习主席书架...,6513,1758,440


In [2]:
# plt.hist(df['like'], bins=np.arange(0,50000,2000), color = 'brown')
# plt.ylabel('number')
# plt.show()

# plt.hist(df['comment'], bins=np.arange(0,30000,2000), color = 'blue')
# plt.ylabel('number')
# plt.show()

# plt.hist(df['forward'], bins=np.arange(0,10000,2000), color = 'red')
# plt.ylabel('number')
# plt.show()

In [130]:
# Split Train Test Data

In [121]:
import scipy
f = open('ml-weibo-project-/tfidf3.txt', 'r')
allsentences = f.readlines()

for i in range(len(allsentences)):
    allsentences[i] = allsentences[i].strip('\n')
# y = df[['like','comment','forward']]
y = np.log(df[['like','forward','comment']])

t = scipy.sparse.coo_matrix(y)
for i, j, v in zip(t.row, t.col, t.data):
    if (np.isnan(v) or np.isinf(v)):
        y.iloc[i, j] = 0

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(allsentences, y, test_size=.3, train_size=.7, random_state=0)

f.close()

<ipython-input-121-dd872dcbf353>:8: RuntimeWarning: divide by zero encountered in log
  y = np.log(df[['like','forward','comment']])


In [122]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(max_features = 10000)
tf_idf_transformer = TfidfTransformer()
tf_idf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(X_train))
X_train_weight = tf_idf.toarray()

tf_idf = tf_idf_transformer.transform(vectorizer.transform(X_test))
X_test_weight = tf_idf.toarray() 


X_train_weight_nn = X_train_weight
y_train_nn = y_train.to_numpy()

X_test_weight_nn = X_test_weight
y_test_nn = y_test.to_numpy()

In [37]:
# print(X_train_weight_nn.shape)
# print(y_train_nn.shape)

In [133]:
# Neural Network

In [134]:
import torch 
from torch.autograd import Variable 
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
  
class Net(torch.nn.Module): 
  def __init__(self, n_feature, n_hidden, n_output): 
    super(Net, self).__init__() 
    self.hidden = torch.nn.Linear(n_feature, n_hidden) 
    self.predict = torch.nn.Linear(n_hidden, n_output) 
  
  def forward(self, x): 
    x = F.relu(self.hidden(x))
    x = self.predict(x) 
    return x 
  
net = Net(10000,60, 3) 
print(net) 
net = net.double()
  
optimizer = torch.optim.SGD(net.parameters(), lr=0.5) 
loss_function = torch.nn.L1Loss()

plt.ion()  
plt.show() 

Net(
  (hidden): Linear(in_features=10000, out_features=60, bias=True)
  (predict): Linear(in_features=60, out_features=3, bias=True)
)


In [ ]:
# Read Data (Dataloader)

In [135]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
import torch.utils.data as Data
import torch

class MyDataSet():
    def __init__(self, inputX, inputY):
        super().__init__()
        self.X = inputX
        self.Y = inputY
        self._len = len(self.X)
        pass
    def __getitem__(self, idx):
        return (self.X[idx], self.Y[idx])
    
    def __len__(self):
        return self._len

In [136]:
batchSize = 256
epochNum = 1
inputSlice = 10000

In [137]:
# train_data:
X_train_weight_nn = X_train_weight_nn.astype(float)
y_train_nn = y_train_nn.astype(float)
train_x = torch.from_numpy(X_train_weight_nn[:inputSlice])
train_y = torch.from_numpy(y_train_nn[:inputSlice])

train_data_set = MyDataSet(train_x, train_y)
train_loader = DataLoader(dataset=train_data_set, batch_size=batchSize, shuffle=True)

# test_data:
X_test_weight_nn = X_test_weight_nn.astype(float)
y_test_nn = y_test_nn.astype(float)
test_x = torch.from_numpy(X_test_weight_nn[:inputSlice])
test_y = torch.from_numpy(y_test_nn[:inputSlice])

test_data_set = MyDataSet(test_x, test_y)
test_loader = DataLoader(dataset=test_data_set, batch_size=batchSize, shuffle=True)

In [48]:
# # train_data:
# X_train_weight_nn = X_train_weight_nn.astype(float)
# y_train_nn = y_train_nn.astype(float)
# train_x = torch.from_numpy(X_train_weight_nn)
# train_y = torch.from_numpy(y_train_nn)

# train_data_set = MyDataSet(train_x, train_y)
# train_loader = DataLoader(dataset=train_data_set, batch_size=batchSize, shuffle=True)

# # test_data:
# X_test_weight_nn = X_test_weight_nn.astype(float)
# y_test_nn = y_test_nn.astype(float)
# test_x = torch.from_numpy(X_test_weight_nn)
# test_y = torch.from_numpy(y_test_nn)

# test_data_set = MyDataSet(test_x, test_y)
# test_loader = DataLoader(dataset=test_data_set, batch_size=batchSize, shuffle=True)

In [ ]:
# Run Model

In [138]:
trainMAE=[]
testMAE=[]
for epoch in range(epochNum):
    # train
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        # run models
        pred_train_y = net(train_x)
        loss = loss_function(pred_train_y, train_y) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        trainMAE.append(loss)
        
    # test    
    for i, data in enumerate(test_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        # run models
        pred_test_y = net(test_x)
        loss = loss_function(pred_test_y, test_y) 
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step() 
        testMAE.append(loss)

In [139]:
print('batchSize:',batchSize, 'epochNum:',epochNum, 'inputSlice:',inputSlice)
pred_train_y_np = pred_train_y.detach().numpy()
train_y_np = train_y.detach().numpy()

train_error = np.absolute(np.subtract(np.exp(train_y_np),np.exp(pred_train_y_np))).mean(axis=0)
#error[i][j]代表第i条text的赞(j=0)转(j=1)评(j=2)的误差
print("TRAIN ERROR!!")
print(train_error)
# print('like: ',train_error.sum(axis=0)[0])
# print('forward: ',train_error.sum(axis=0)[1])
# print('comment: ',train_error.sum(axis=0)[2])

pred_test_y_np = pred_test_y.detach().numpy()
test_y_np = test_y.detach().numpy()

test_error = np.absolute(np.subtract(np.exp(test_y_np),np.exp(pred_test_y_np))).mean(axis=0)
#error[i][j]代表第i条text的赞(j=0)转(j=1)评(j=2)的误差
print("TEST ERROR!!")
print(test_error)
# print('like: ',test_error.sum(axis=0)[0])
# print('forward: ',test_error.sum(axis=0)[1])
# print('comment: ',test_error.sum(axis=0)[2])

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12536.09880779  1255.29824634  5439.73813139]
TEST ERROR!!
[12441.87213662  1333.97634073  9424.12400238]


In [133]:
# neuron:60

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12613.8147136   1293.52379389  5489.95362087]
TEST ERROR!!
[12488.83993737  1352.77439202  9449.21560023]


In [128]:
# neuron:60

batchSize: 256 epochNum: 1 inputSlice: 1000
TRAIN ERROR!!
[13940.55073041  1463.01430491  2813.44433646]
TEST ERROR!!
[15956.80022126  1858.11942512  4257.14093823]


In [76]:
# neuron:5

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12737.46688887  1334.04577996  5574.20811499]
TEST ERROR!!
[12573.91204917  1375.28546174  9504.5288857 ]


In [88]:
# neuron:7

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12631.66932127  1349.04199398  5548.54195958]
TEST ERROR!!
[12509.48492121  1458.41987516  9586.88086884]


In [104]:
# neuron:10

batchSize: 512 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12568.35569375  1289.12045663  5485.02483937]
TEST ERROR!!
[12435.42117523  1338.55812754  9433.197557  ]


In [69]:
# neuron:10

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12527.43855916  1254.24672193  5437.73017198]
TEST ERROR!!
[12307.77097221  1302.55125891  9360.99570437]


In [100]:
# neuron:10

batchSize: 128 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12456.39723128  1246.77121704  5418.0303683 ]
TEST ERROR!!
[12369.86210662  1320.58609568  9394.37296849]


In [94]:
# neuron:15

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12553.71922625  1262.64620268  5450.06649833]
TEST ERROR!!
[12461.96738681  1339.15609752  9433.16608345]


In [63]:
# neuron:20

batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
[12662.9252431   1352.55885474  5574.63375898]
TEST ERROR!!
[12505.0915534   1414.75197035  9553.57151403]


In [50]:
# neuron:10

batchSize: 256 epochNum: 10 inputSlice: no slice
TRAIN ERROR!!
[12202.86462236  1246.54009916  5156.61817217]
TEST ERROR!!
[12346.21355304  1144.47399722  5054.57358725]


In [46]:
# neuron:10

batchSize: 256 epochNum: 1 inputSlice: no slice
TRAIN ERROR!!
[12715.42325212  1374.7204235   5521.23894605]
TEST ERROR!!
[12300.36565613  1293.80394025  5431.11605509]


batchSize: 128 epochNum: 1 inputSlice: 20000
TRAIN ERROR!!
[12990.45075936  1345.25226159  5862.56904238]
TEST ERROR!!
like:  257353618.72064707
forward:  26217073.951993212
comment:  136723599.79981777


tensor(1.2589, dtype=torch.float64, grad_fn=<L1LossBackward>) tensor(7.0656, dtype=torch.float64, grad_fn=<L1LossBackward>)
[]
[13134.86642161  1317.44547583  5836.74023574]


array([[7.5501395 , 6.17409849, 6.83437125],
       [7.42997805, 6.07832344, 6.72076686],
       [7.51336999, 6.14377439, 6.80254086],
       ...,
       [7.1439601 , 5.90179181, 6.52287035],
       [7.25993323, 5.95999596, 6.59218973],
       [7.14575197, 5.90507022, 6.53357377]])

In [159]:
# hidden_layer = 20?

batchSize: 128 epochNum: 1 inputSlice: 20000
TRAIN ERROR!!
[13134.86642161  1317.44547583  5836.74023574]
TEST ERROR!!
like:  252344190.9646321
forward:  26419602.406009596
comment:  136715710.59653315


In [152]:
# hidden_layer = 10

batchSize: 128 epochNum: 1 inputSlice: 20000
TRAIN ERROR!!
[13145.15758016  1323.63056136  5847.76483202]
TEST ERROR!!
like:  252636293.21363625
forward:  26601557.223065767
comment:  136887992.18186474


batchSize: 256 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
like:  125277879.4885846
forward:  12545197.052814486
comment:  54410631.49697019
TEST ERROR!!
like:  122896861.8088555
forward:  12881456.964935735
comment:  93452552.88057627


batchSize: 128 epochNum: 1 inputSlice: 10000
TRAIN ERROR!!
like:  126571200.22719258
forward:  13117773.349215588
comment:  55238182.09323303
TEST ERROR!!
like:  121946858.26988766
forward:  13547522.250401475
comment:  94217450.57214229


In [120]:
# batchsize:128  epoch:1  input:all 

TRAIN ERROR!!
like:  120357282.37726352
forward:  12524944.590262374
comment:  53188217.36421586
TEST ERROR!!
like:  120946834.02902414
forward:  12795199.895387985
comment:  92626643.96062796


In [1]:
# epoch:1 input:1000 batchsize:256 